In [1]:
import pandas as pd
import numpy as np
import requests, copy
from pprint import pprint
from bs4 import BeautifulSoup

### Create Initial DataFrame from scraping http://www.prosportstransactions.com/

In [7]:
df_json = pd.read_json("../data/pst_nba_injuries_2020_2021.txt")
df = df_json.copy()
df.head(5)

,Date,Team,Healed,Injured,Notes
0,2020-03-11,76ers,Joel Embiid,,activated from IL
1,2020-03-11,Bucks,Eric Bledsoe,,activated from IL
2,2020-03-11,Bucks,George Hill,,activated from IL
3,2020-03-11,Bulls,,Chandler Hutchison,surgery on right shoulder (out for season)
4,2020-03-11,Heat,,Jimmy Butler,placed on IL with left big toe injury


### Reformat the DataFrame

In [8]:

df['Player'] = df['Healed'] + df['Injured']
df['Status'] = np.where(df['Healed'] == '', 'Injured', 'Healed')
df = df[['Date', 'Team', 'Player', 'Status', 'Notes']]
df.head(10)

,Date,Team,Player,Status,Notes
0,2020-03-11,76ers,Joel Embiid,Healed,activated from IL
1,2020-03-11,Bucks,Eric Bledsoe,Healed,activated from IL
2,2020-03-11,Bucks,George Hill,Healed,activated from IL
3,2020-03-11,Bulls,Chandler Hutchison,Injured,surgery on right shoulder (out for season)
4,2020-03-11,Heat,Jimmy Butler,Injured,placed on IL with left big toe injury
5,2020-03-11,Heat,Jae Crowder,Healed,returned to lineup
6,2020-03-11,Heat,Tyler Herro,Healed,activated from IL
7,2020-03-11,Hornets,Terry Rozier,Injured,placed on IL with illness
8,2020-03-11,Jazz,Emmanuel Mudiay,Injured,illness (DTD)
9,2020-03-11,Jazz,Rudy Gobert,Injured,COVID-19 coronavirus (out indefinitely)


### Extract players out for COVID or "Health and Safety Protocols"

In [9]:
df_cv19 = df[np.logical_or(df['Notes'].str.contains('COVID'), df['Notes'].str.contains('protocol'))]
df_cv19

,Date,Team,Player,Status,Notes
9,2020-03-11,Jazz,Rudy Gobert,Injured,COVID-19 coronavirus (out indefinitely)
19,2020-03-12,Jazz,Donovan Mitchell,Injured,COVID-19 coronavirus (DTD)
21,2020-03-14,Pistons,Christian Wood,Injured,COVID-19 coronavirus (DTD)
22,2020-03-17,Nets,Kevin Durant,Injured,NBA health and safety protocols (DTD)
24,2020-03-19,Celtics,Marcus Smart,Injured,COVID-19 coronavirus (out indefinitely)
...,...,...,...,...,...
2846,2021-05-09,Pistons,Hamidou Diallo,Injured,placed on IL with NBA health and safety protocols
2957,2021-05-14,Thunder,Josh Hall,Injured,placed on IL with NBA health and safety protoc...
2969,2021-05-15,Heat,Omer Yurtseven,Injured,placed on IL with NBA health and safety protocols
3031,2021-05-16,Spurs,DaQuan Jeffries,Injured,placed on IL with NBA health and safety protoc...


240 instances of players being held out due to COVID protocols since 03/11/2020 (The day Rudy Gobert tested positive and the league shut down)

### Just COVID data for 2020-2021 NBA Season

In [10]:
df_cv19_2021 = df_cv19[df_cv19['Date'] >= '12-01-2020']
df_cv19_2021

,Date,Team,Player,Status,Notes
415,2020-12-22,Blazers,Nassir Little,Injured,COVID-19 (DTD)
423,2020-12-22,Hawks,Rajon Rondo,Injured,placed on IL with NBA health and safety protocols
468,2020-12-23,Rockets,Kenyon Martin Jr. / K.J. Martin,Injured,placed on IL with NBA health and safety protocols
472,2020-12-25,Bucks,Mamadi Diakite,Injured,placed on IL with NBA health and safety protoc...
475,2020-12-25,Rockets,Ben McLemore,Injured,COVID-19 protocols
...,...,...,...,...,...
2846,2021-05-09,Pistons,Hamidou Diallo,Injured,placed on IL with NBA health and safety protocols
2957,2021-05-14,Thunder,Josh Hall,Injured,placed on IL with NBA health and safety protoc...
2969,2021-05-15,Heat,Omer Yurtseven,Injured,placed on IL with NBA health and safety protocols
3031,2021-05-16,Spurs,DaQuan Jeffries,Injured,placed on IL with NBA health and safety protoc...


230 instances of players being held out due to COVID protocols during 2020-2021 NBA season

### Unique Players

In [15]:
players = df_cv19_2021['Player'].unique()
len(players)

182

182 Unique Players were held out due to COVID protocols during 2020-2021 NBA season

### Generate basketball-reference.com IDs

In [18]:
bbref_id_list = []
for player in players:
    player_copy = player
    # cleaning name form
    if '.' in player:
        player_copy = player_copy.replace('.', '')
    if 'ö' in player:
        player_copy = player_copy.replace('ö', 'o')
    if 'Bam Adebayo' in player_copy:
        player_copy = 'Bam Adebayo'
    
    player_split = player_copy.lower().split()
    
    # Typical ID format
    bbref_id = player_split[1][:5] + player_split[0][:2] + '01'
    # Special Cases that threw errors in test
    if player_split[1] == 'kleber':
        bbref_id = 'klebima01'
    if player_split[1] == 'ntilikina':
        bbref_id = 'ntilila01'
    bbref_id_list.append([player, bbref_id])

    ### Test to check if ID's are correct
    # url = f'https://www.basketball-reference.com/players/{bbref_id[0]}/{bbref_id}.html'
    # r = requests.get(url)
    # if r.status_code == 200:
    #     print(player, bbref_id, r.status_code)
    # else:
    #     print(player.upper())

In [19]:
df_bbref_id = pd.DataFrame(bbref_id_list, columns=['Player', 'BBRef_ID'])

### Merging BBRef IDs with DataFrame

In [20]:
df_main = pd.merge(left=df_cv19_2021, right=df_bbref_id, how='left', left_on=df_cv19_2021.Player, right_on=df_bbref_id.Player)
df_main

,key_0,Date,Team,Player_x,Status,Notes,Player_y,BBRef_ID
0,Nassir Little,2020-12-22,Blazers,Nassir Little,Injured,COVID-19 (DTD),Nassir Little,littlna01
1,Rajon Rondo,2020-12-22,Hawks,Rajon Rondo,Injured,placed on IL with NBA health and safety protocols,Rajon Rondo,rondora01
2,Kenyon Martin Jr. / K.J. Martin,2020-12-23,Rockets,Kenyon Martin Jr. / K.J. Martin,Injured,placed on IL with NBA health and safety protocols,Kenyon Martin Jr. / K.J. Martin,martike01
3,Mamadi Diakite,2020-12-25,Bucks,Mamadi Diakite,Injured,placed on IL with NBA health and safety protoc...,Mamadi Diakite,diakima01
4,Ben McLemore,2020-12-25,Rockets,Ben McLemore,Injured,COVID-19 protocols,Ben McLemore,mclembe01
...,...,...,...,...,...,...,...,...
225,Hamidou Diallo,2021-05-09,Pistons,Hamidou Diallo,Injured,placed on IL with NBA health and safety protocols,Hamidou Diallo,diallha01
226,Josh Hall,2021-05-14,Thunder,Josh Hall,Injured,placed on IL with NBA health and safety protoc...,Josh Hall,halljo01
227,Omer Yurtseven,2021-05-15,Heat,Omer Yurtseven,Injured,placed on IL with NBA health and safety protocols,Omer Yurtseven,yurtsom01
228,DaQuan Jeffries,2021-05-16,Spurs,DaQuan Jeffries,Injured,placed on IL with NBA health and safety protoc...,DaQuan Jeffries,jeffrda01
